First, load the data, from the supplied data file

In [1]:
import tarfile
import json
import gzip
import pandas as pd
import botometer
from pandas.io.json import json_normalize

## VARIABLE INITIATION
tar = tarfile.open("../input/2017-09-22.tar.gz", "r:gz")
mashape_key = "QRraJnMT9KmshkpJ7iu74xKFN1jtp1IyBBijsnS5NGbEuwIX54"
twitter_app_auth = {
    'consumer_key': 'sPzHpcj4jMital75nY7dfd4zn',
    'consumer_secret': 'rTGm68zdNmLvnTc22cBoFg4eVMf3jLVDSQLOwSqE9lXbVWLweI',
    'access_token': '4258226113-4UnHbbbxoRPz10thy70q9MtEk9xXfJGOpAY12KW',
    'access_token_secret': '549HdasMEW0q2uV05S5s4Uj5SdCeEWT8dNdLNPiAeeWoX',
  }
bom = botometer.Botometer(wait_on_ratelimit=True,
                          mashape_key=mashape_key,
                          **twitter_app_auth)
count = 0

In [2]:
data = pd.DataFrame()
uname = pd.DataFrame()
count=0
#uname = []
for members in tar.getmembers():
    #if (None):
       # break
    #else:
    if (count==13):
        f = tar.extractfile(members)
        data = data.append(pd.read_json(f, lines=True))
        for memberx in data['user']:
            uname=uname.append(json_normalize(memberx)['screen_name'], ignore_index=True)
            #uname.append('@'+str(json_normalize(memberx)['screen_name'].values[0]))
    count = count + 1

In [4]:
len(uname)

4369

In [5]:
distinct_uname=[]
for i in uname.drop_duplicates().values:
    distinct_uname.append((str('@'+i).replace("[u'","")).replace("']",''))

In [7]:
len(distinct_uname)
#asu=distinct_uname[0:180]

3231

In [ ]:
botoresult = pd.DataFrame()

for screen_name, result in bom.check_accounts_in(asu):
    botoresult=botoresult.append(result, ignore_index=True)

In [94]:
#bom.twitter_api.rate_limit_status()['resources']['application']['/application/rate_limit_status']['remaining']

In [104]:
output_bot=pd.concat([botoresult.user.apply(pd.Series), botoresult.scores.apply(pd.Series), botoresult.categories.apply(pd.Series)], axis=1)

In [103]:
len(botoresult)

1112

In [453]:
output_bot.to_csv("outputbot.csv", sep=',', encoding='utf-8')